In [1]:
using Pkg
Pkg.activate("Dev")
using Revise
using Skylight
using CairoMakie

  Activating project at `~/repos/Skylight.jl/run/Dev`


In [52]:
distance = 500
N = 20
spacetime = KerrSpacetimeBoyerLindquistCoordinates(M=1.0,a=0.6)
camera = PinholeCamera(position = [0.0, distance, π/2-π/20, 0.0],
                        horizontal_aperture_in_degrees = rad2deg(315/distance),
                        vertical_aperture_in_degrees = rad2deg(315/distance),
                        horizontal_number_of_pixels = N,
                        vertical_number_of_pixels = N)
model = IonTorus()
torus_specific_angular_momentum!(model, spacetime)
cusp_and_center_radius!(model, spacetime)
torus_potentials_at_center_and_surface!(model, spacetime)
configurations = NonVacuumOTEConfigurations(spacetime=spacetime,
                                   camera = camera,
                                   radiative_model = model,
                                   unit_mass_in_solar_masses=1.0,
                                   observation_energies = [1.0])
initial_data = initialize(configurations)
cb, cbp = callback_setup(configurations; rhorizon_bound=2e-3) #... or, define your own cb and cbp
run = integrate(initial_data, configurations, cb, cbp; method=VCABM(), reltol=1e-5, abstol=1e-5)
output_data = run.output_data

Equations integration stats:
Wall clock time: 0.277612042 seconds
Memory allocated: 81.741 MiB


10×400 Matrix{Float64}:
 -975.737        -983.841        …  -983.897        -975.634
  550.0           550.0              550.0           550.0
    2.23295         2.24158            1.16981         1.16712
   -2.47664        -2.54642            2.64466         2.58338
   -1.00164        -1.00164           -1.00164        -1.00164
    0.926036        0.93349      …     0.933728        0.926077
   -0.000339642    -0.000366639        0.000459427     0.000452213
   -0.000738556    -0.000672416        0.000486322     0.000543706
    0.0             0.0                0.0             0.0
    0.0             0.0                0.0             0.0

In [32]:
# position = camera.position
position = [0.0, model.rcenter, π/2, 0.0]
# Skylight.torus_normalized_potential(position, spacetime, model)
# Skylight.energy_density(position, spacetime, model)

4-element Vector{Float64}:
 0.0
 5.842026702136471
 1.5707963267948966
 0.0

In [62]:
g = zeros(4,4)
v = zeros(4)
function is_inside_torus(r, spacetime, model, g)
    coords_top = coordinates_topology(spacetime)
    position = Skylight.equatorial_position(r, coords_top)
    metric!(g, position, spacetime)
    return Skylight.torus_normalized_potential(g, model) > 0.0
end
function is_rest_frame_timelike(r, spacetime, model, g, v)
    coords_top = coordinates_topology(spacetime)
    position = Skylight.equatorial_position(r, coords_top)
    metric!(g, position, spacetime)
    rest_frame_four_velocity!(v, position, g, spacetime, model, coords_top)
    return Skylight.is_timelike(v, g)
end

radii = model.rcusp:0.1:100.0

tim = map(r -> is_rest_frame_timelike(r, spacetime, model, g, v), radii)
pot = map(r -> is_inside_torus(r,spacetime,model,g), radii)
# rest_frame_energy = -scalar_product(vμ, momentum, metric)
# ε .= observation_energies*rest_frame_energy


973-element Vector{Bool}:
 0
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [56]:
all(tim)

true

In [41]:
Skylight.is_timelike(v,g)

true

In [43]:
k = [-1.0,0.0,0.0,0.0]

Skylight.normalize_timelike!(k, g)

In [46]:
Skylight.scalar_product(v,k,g)

1.1420768862604598

In [ ]:
xs, ys = axes_ranges(camera) 
zs = grid_view(output_data, configurations; energy_index = 1)

fig = Figure(font = "CMU Serif") #resolution=(600,400)
ax = Axis(fig[1,1], xlabel=L"\alpha", ylabel=L"\beta", ylabelsize = 26, xlabelsize = 26) 
hmap = heatmap!(xs, ys, zs; colormap=:gist_heat, interpolate=true)
Colorbar(fig[:, end+1], hmap, label=L"I", labelsize=26, width = 15, ticksize = 18, tickalign = 1)
colsize!(fig.layout, 1, Aspect(1, 1.0))
colgap!(fig.layout, 7)
CairoMakie.save("plot.png", fig)